NLP - or Natural Language Processing ou processamento de linguagem natural é de uma forma simples é um conjunto amplo de técnica desenvolvidas para ajudar as máquinas a aprender textos. A NLP da vida de chatbots a motores de busca e é usado para diversas tarefas como análise de sentimentos e tradução. 


In [4]:
#IMPORTANDO LIBS
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing


In [5]:
# IMPORTANDO DADOS
train_df = pd.read_csv("input/nlp-getting-start/train.csv")
test_df = pd.read_csv("input/nlp-getting-start/test.csv")


In [8]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


Exemplo de tweets normais

In [9]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [11]:
train_df[train_df["target"] == 0]["text"].values[0:5]

array(["What's up man?", 'I love fruits', 'Summer is lovely',
       'My car is so fast', 'What a goooooooaaaaaal!!!!!!'], dtype=object)

Exemplo de um tweet de desastre:

In [12]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

## Construindo vetores
A teoria por trás deste modelo é bem simples: as palavras contendo cada tweet são bons indicadores se eles são reais desastres ou não. (Não está inteiramente correto mas é um bom lugar pra começar)

Para isso podemos usar a biblioteca CountVectorizer do scikit-learn para contar as palavras em cada tweet e transformá-las em dados nosso modelo de machine learning possa ser processado. 

O vetor é nesse contexto, é um conjunto de numeros que o modelo de machine learning pode trabalhar. 



In [14]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [16]:
print(example_train_vectors)

  (0, 34)	1
  (0, 12)	1
  (0, 5)	1
  (0, 49)	1
  (0, 39)	1
  (0, 29)	1
  (0, 50)	1
  (0, 13)	1
  (0, 25)	1
  (0, 4)	1
  (0, 18)	1
  (0, 52)	1
  (0, 3)	1
  (1, 17)	1
  (1, 16)	1
  (1, 26)	1
  (1, 24)	1
  (1, 42)	1
  (1, 44)	1
  (1, 11)	1
  (2, 5)	2
  (2, 3)	1
  (2, 41)	1
  (2, 7)	1
  (2, 51)	1
  :	:
  (2, 32)	1
  (2, 15)	1
  (3, 21)	1
  (3, 14)	1
  (3, 32)	1
  (3, 1)	1
  (3, 0)	1
  (3, 35)	1
  (3, 40)	1
  (3, 53)	1
  (3, 10)	1
  (4, 50)	1
  (4, 53)	1
  (4, 23)	1
  (4, 20)	1
  (4, 46)	1
  (4, 36)	1
  (4, 19)	2
  (4, 43)	1
  (4, 2)	1
  (4, 6)	1
  (4, 48)	1
  (4, 38)	1
  (4, 22)	1
  (4, 45)	1


In [20]:
## usar o .todense() pois os vetores são  enxutos apenas elementos diferente de zero são salvos.
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


A informação acima mostra que, há 54 palavras únicas (ou tokens) nos primeiros 5 tweets. 

O primeiro tweet contem apenas alguns desses tokens únicos, todos os contadores que não são zero acima são tokes que existem no primeiro tweet. Agora vamos criar um vetor para todos os tweets.


In [22]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

## Nosso modelo

Como mencionado antes, podemos pensar nas palavras contidas em cada tweet são bons indicadores se eles são sobre desastres reais ou não. A presença de uma palavra em particular (ou o conjunto de palavras) em um tweet deve estar relacionado diretamente se o tweet é real ou não. 

O que estamos assumindo aqui é uma correlação linear, então vamos construir um modelo linear e ver.



In [23]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

Vamos testar nosso modelo e ver quão bem ele está em relação aos dados de treino. Para isso, vamos usar a cross-validation, onde treinamos parte dos dados conhecidos e validamos com o restante. Se fizermos isso algumas vezes (com porções diferentes) podemos ter uma boa ideia de como um modelo particular ou método performa. 

A métrica para esta competição é F1, então vamos usá-la aqui.



In [26]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores


array([0.59421842, 0.56498283, 0.64113893])

Os scores acimas não são terríveis! Parecessem que assumimos um score próximo a 0.65 no leaderboard. Existem várias formas de potencializar e melhorar isso (TFIDF, LSA, LSTM / RNNs, ..) - Faça um teste!

Neste meio tempo, faremos previsões no nosso conjunto de treino e vamos construir dados de envio para a competição. 


In [27]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [28]:
sample_submission = pd.read_csv("input/nlp-getting-start/sample_submission.csv")

In [29]:
sample_submission["target"] = clf.predict(test_vectors)


In [30]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [34]:
sample_submission.to_csv("submission.csv", index=False)

Tentar melhorar os dados

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer()
vectorizer.fit_transform([train_vectors, train_df["text"]])

AttributeError: lower not found